<a href="https://colab.research.google.com/github/Ajay-user/DataScience/blob/master/Natural%20Language%20Processing%20/Bidirectional_LSTM_on_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf

## Constants

In [2]:

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

## Build the model

In [3]:
# Input for variable-length sequences of integers
inputs = tf.keras.layers.Input(shape=(None,), dtype='int32')
# Embed each integer in a 128-dimensional vector
embedding = tf.keras.layers.Embedding(max_features+1, 128)(inputs)
# add two bidirectional lstm
bidirectional_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(embedding)
bidirectional_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(bidirectional_lstm)
# add output classifier
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(bidirectional_lstm)

model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         2560128   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,757,889
Trainable params: 2,757,889
Non-trainable params: 0
_________________________________________________________________



## Load the IMDB movie review sentiment data
https://keras.io/api/datasets/imdb/

This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a list of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

*arguments<br>
* maxlen: int or None. Maximum sequence length. Any longer sequence will be truncated. Defaults to None, which means no truncation.

* num_words: integer or None. Words are ranked by how often they occur (in the training set) and only the num_words most frequent words are kept. Any less frequent word will appear as oov_char value in the sequence data. If None, all words are kept. Defaults to None, so all words are kept.

In [4]:
(X_train, y_train),(X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=max_features)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [5]:
print('Shape of training data', X_train.shape)
print('Shape of training label', y_train.shape)
print('Shape of testing data', X_test.shape)
print('Shape of testing label', y_test.shape)

Shape of training data (25000,)
Shape of training label (25000,)
Shape of testing data (25000,)
Shape of testing label (25000,)


In [6]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen)

## Train and evaluate the model

In [7]:
model.compile(optimizer='adam', loss ='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

782/782 [==============================] - 418s 526ms/step - loss: 0.3943 - accuracy: 0.8217 - val_loss: 0.3512 - val_accuracy: 0.8559


In [8]:
# model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss', loss)
print('Accracy',accuracy)

782/782 [==============================] - 70s 90ms/step - loss: 0.3512 - accuracy: 0.8559
Loss 0.35116586089134216
Accracy 0.8559200167655945
